<a href="https://colab.research.google.com/github/duc-ke/MONAI_examples/blob/main/sp3_st6_inference_FashionMNIST_igniteList-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks for Classifying Fashion-MNIST Dataset using Ignite
This is a tutorial on using Ignite to train neural network models, setup experiments and validate models.

In this notebook, we will be doing classification of images using Convolutional Neural Networks 

We will be using the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) Fashion-MNIST is a set of 28x28 grayscale images of clothes.

![Fashion MNIST dataset](https://github.com/abdulelahsm/ignite/blob/update-tutorials/examples/notebooks/assets/fashion-mnist.png?raw=1)

Lets get started!


----
28x28 grayscale 이미지

## Required Dependencies

We assume that `torch` and `ignite` are already installed. We can install it using `pip`:

In [5]:
!pip install pytorch-ignite

     |████████████████████████████████| 251 kB 5.3 MB/s 


### Importing libraries

General Data-Science Libraries like numpy, matplotlib and seaborn

In [ ]:
# !pip install seaborn

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

We import `torch`, `nn` and `functional` modules to create our models.

We also import `datasets` and `transforms` from torchvision for loading the dataset and applying transforms to the images in the dataset.

We import `Dataloader` for making train and validation loader for loading data into our model.

In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
torch.__version__

'1.10.0+cu111'

`Ignite` is a High-level library to help with training neural networks in PyTorch. It comes with an `Engine` to setup a training loop, various metrics, handlers and a helpful contrib section! 

Below we import the following:
* **Engine**: Runs a given process_function over each batch of a dataset, emitting events as it goes.(주어진 process_function을 실행하여 이벤트를 발생)
* **Events**: Allows users to attach functions to an `Engine` to fire functions at a specific event. Eg: `EPOCH_COMPLETED`, `ITERATION_STARTED`, etc.
* **Accuracy**: Metric to calculate accuracy over a dataset, for binary, multiclass, multilabel cases. 
* **Loss**: General metric that takes a loss function as a parameter, calculate loss over a dataset.
* **RunningAverage**: General metric to attach to Engine during training. 
* **ModelCheckpoint**: Handler to checkpoint models. 
* **EarlyStopping**: Handler to stop training based on a score function. 

In [4]:
## trainer, evaluator를 create 에서 가져와서 씀
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping

ModuleNotFoundError: ignored

In [ ]:
from ignite.contrib.handlers import ProgressBar

The code below first sets up transform using `torhvision transfroms` for converting images to pytorch tensors and normalizing the images.

Next, We use `torchvision datasets` for dowloading the fashion mnist dataset and applying transforms which we defined above.

* `trainset` contains the training data.
* `validationset` contains the validation data

Next, We use `pytorch dataloader` for making dataloader from the train and validation sets.

-----
torchvision transform : 이미지를 텐서 변환, 정규화

In [ ]:
# !pip install ipywidgets

In [ ]:
# transform to normalize the data
# gray 이미지기 때문에 1 채널 밖에 없음
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = datasets.FashionMNIST('./data', download=True, train=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
validationset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform)
val_loader = DataLoader(validationset, batch_size=64, shuffle=True)

In [ ]:
aaa = next(iter(train_loader))
len(aaa)   # img와 label 두개임, dict form 이였다면 aaa에 aaa['img'], aaa['seg']로 접근했어야 함

2

In [ ]:
# 흑백이미지라서 Channel이 1임
aaa[0].shape, aaa[1].shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

### CNN Model

Explanation of Model Architecture

* [Convolutional layers](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), the Convolutional layer is used to create a convolution kernel that is convolved with the layer input to produce a tensor of outputs.
* [Maxpooling layers](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html), the Maxpooling layer is used to downsample an input representation keeping the most active pixels from the previous layer.
* The usual [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) + [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html) layers to avoid overfitting and produce a 10-dim output.
* We had used [Relu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) Non Linearity for the model and [logsoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html) at the last layer because we are going to use the [NLLL loss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html).


In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.convlayer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.convlayer2 = nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(64*6*6,600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)   # 마지막 출력을 10개로
        
    def forward(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = x.view(-1,64*6*6)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.fc3(x)   
        
        return F.log_softmax(x,dim=1)   # log_softmax로 10개 확률값 취하기(multiclasses classification적용)

### Creating Model, Optimizer and Loss

Below we create an instance of the CNN model. The model is placed on a device and then a loss function of `negative log likelihood loss` and `Adam optimizer` with learning rate of 0.001 are setup. 

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = '6'

In [ ]:
# creating model,and defining optimizer and loss
model = CNN()
# moving model to gpu if available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()    # negative log likelihood loss. 
# 만약 crossentropy loss를 사용하면 마지막 레이어에 log_softmax를 사용안해도 됌.

### Training and Evaluating using Ignite

### Instantiating Training and Evaluating Engines

Below we create 3 engines, a trainer, an evaluator for the training set and an evaluator for the validation set, by using the `create_supervised_trainer` and `create_supervised_evaluator` and passing the required arguments.

We import the metrics from `ignite.metrics` which we want to calculate for the model. Like `Accuracy`, `ConfusionMatrix`, and `Loss` and we pass them to `evaluator` engines which will calculate these metrics for each iteration.

* `training_history`: it stores the training loss and accuracy
* `validation_history`:it stores the validation loss and accuracy
* `last_epoch`: it stores the last epoch untill the model is trained


----
* create_supvervised.. 를 쓰면 trainer, train set+vadiation set에 대한 evaluator 3가지를 쉽게 얻을 수 있다.
* metrics로 accuracy, confusionmatrix, loss를 가져옴.
* 그외 변수 3가지를 추가로 생성하여 history 기록
  * training_history : loss, accuracy저장, 
  * val~~ : loss, accuracy 저장
  * last_epoch : 마지막 epoch를 저장

## 중요
1. evaluator를 validation만 하지말고 train evaluator를 생성하면 train loss뿐 아니라 metrics를 따로 기록할 수 있다!!!!
2. 또한 pytorch 로 직접 짜지 않아도 loss 와 metric을 저장할 변수를 만들어서 custom logging을 trainer에 붙일 수 있다.

In [ ]:
# defining the number of epochs
epochs = 12
# creating trainer,evaluator
# dict form이 아니라서 prepare batch 함수를 따로 전달할 필요가 없다.
""" 예시) dict form 인경우 인덱스 접근이 불가하므로 직접 key를 이용하여 찢어서 넣어줘야 한다.
def prepare_batch(batch, device=None, non_blocking=False):
    return _prepare_batch((batch["img"], batch["seg"]), device, non_blocking)


## Ignite - create_supervised_trainer
trainer = create_supervised_trainer(
    net, opt, loss, device, False, prepare_batch=prepare_batch
)
"""
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

## engine.state.metrics에 아래의 키값들이 저장됌.
metrics = {
    'accuracy':Accuracy(),
    'nll':Loss(criterion),
    'cm':ConfusionMatrix(num_classes=10)
}
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
training_history = {'accuracy':[],'loss':[]}
validation_history = {'accuracy':[],'loss':[]}
last_epoch = []

### Metrics - RunningAverage

To start, we will attach a metric of `RunningAverage` to track a running average of the scalar loss output for each batch. 

In [ ]:
# 이걸 뺴면 어떻게 될지 확인해보자.!!!!!!!!!!!!!!
# engine's state metrics['loss']에 값을 넣게 된다. -> progressbar에서 추적
RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

In [ ]:
# Ignite의 progress bar를 trainer에 연결, metrics 키를 전달. 이렇게 되면 진행바는 engine.state.metrics['loss']를 추적함
pbar = ProgressBar(persist=True, bar_format="")
pbar.attach(trainer, ['loss'])

### EarlyStopping - Tracking Validation Loss

Now we will setup a `EarlyStopping` handler for this training process. EarlyStopping requires a score_function that allows the user to define whatever criteria to stop trainig. In this case, if the loss of the validation set does not decrease in 10 epochs, the training process will stop early. Since the `EarlyStopping` handler relies on the validation loss, it's attached to the `val_evaluator`. 

In [ ]:
def score_function(engine):
    val_loss = engine.state.metrics['nll']
    return -val_loss   # -를 붙이면 낮을 수록 좋음

handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, handler)

### Attaching Custom Functions to Engine at specific Events

Below you will see ways to define your own custom functions and attaching them to various `Events` of the training process.

The functions below both achieve similar tasks, they print the results of the evaluator run on a dataset. One function does that on the training evaluator and dataset, while the other on the validation. Another difference is how these functions are attached in the trainer engine.

The first method involves using a decorator, the syntax is simple - `@` `trainer.on(Events.EPOCH_COMPLETED)`, means that the decorated function will be attached to the trainer and called at the end of each epoch. 

The second method involves using the add_event_handler method of trainer - `trainer.add_event_handler(Events.EPOCH_COMPLETED, custom_function)`. This achieves the same result as the above. 

**커스텀 log 출력함수를 만든다.**

In [ ]:
# log 출력 함수
# training시, train evaluator의 loss, accuracy + validation evluateor의 loss, accuracy 출력

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    accuracy = metrics['accuracy']*100
    loss = metrics['nll']
    last_epoch.append(0)    # last_epoch.append(trainer.state.epoch) 아닌가?
    training_history['accuracy'].append(accuracy)  # 이렇게 중간 결과를 저장가능하다!
    training_history['loss'].append(loss)
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, accuracy, loss))

def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    accuracy = metrics['accuracy']*100
    loss = metrics['nll']
    validation_history['accuracy'].append(accuracy)
    validation_history['loss'].append(loss)
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, accuracy, loss))
    
trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)    

### Confusion Matrix

Confusion matrix gives us a better idea of what our classification model is getting right and what types of errors it is making.

We visualize the `confusion matrix` using the `seaborn.heatmap` from `seaborn` library.

----
seaborn의 heatmap으로 시각화

In [ ]:

# validation evaluator에 연결

@trainer.on(Events.COMPLETED)
def log_confusion_matrix(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    cm = metrics['cm']
    cm = cm.numpy()
    cm = cm.astype(int)
    classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']
    fig, ax = plt.subplots(figsize=(10,10))  
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt="d")
    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix') 
    ax.xaxis.set_ticklabels(classes,rotation=90)
    ax.yaxis.set_ticklabels(classes,rotation=0)

### ModelCheckpoint

Lastly, we want to checkpoint this model. It's important to do so, as training processes can be time consuming and if for some reason something goes wrong during training, a model checkpoint can be helpful to restart training from the point of failure.

Below we will use Ignite's `ModelCheckpoint` handler to checkpoint models at the end of each epoch.

In [ ]:
checkpointer = ModelCheckpoint('./models', 'fashionMNIST', n_saved=2, create_dir=True, save_as_state_dict=True, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'fashionMNIST': model})

### Run Engine

Next, we will run the trainer for 12 epochs and monitor results. Below we can see that custom functions defined above helps prints the `loss` and `accuracy` per epoch.  


In [ ]:
torch.__version__

'1.10.0a0+0aef44c'

In [ ]:
trainer.run(train_loader, max_epochs=epochs)

Current run is terminating due to exception: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
Engine run is terminating due to exception: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

### Plotting the loss and accuracy
Next, we will plot the loss and accuracy which we have stored in the `training_history` and `validation_history` dictionary to see how loss and accuracy are changing with each epoch.

In [ ]:
plt.plot(training_history['accuracy'],label="Training Accuracy")
plt.plot(validation_history['accuracy'],label="Validation Accuracy")
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(frameon=False)
plt.show()

In [ ]:
plt.plot(training_history['loss'],label="Training Loss")
plt.plot(validation_history['loss'],label="Validation Loss")
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.legend(frameon=False)
plt.show()

### Loading the saved model from the disk
Loading the saved pytorch model from the disk for inferencing.

----

모델 껍데기에 pickle파일 load만하면 됌.

In [ ]:
## 맨 마지막꺼 로드함.
# loading the saved model
def fetch_last_checkpoint_model_filename(model_save_path):
    import os
    checkpoint_files = os.listdir(model_save_path)
    checkpoint_files = [f for f in checkpoint_files if '.pt' in f]
    checkpoint_iter = [
        int(x.split('_')[2].split('.')[0])
        for x in checkpoint_files]
    last_idx = np.array(checkpoint_iter).argmax()
    print(checkpoint_files)
    print(last_idx)
    print(checkpoint_files[last_idx])
    return os.path.join(model_save_path, checkpoint_files[last_idx])

model.load_state_dict(torch.load(fetch_last_checkpoint_model_filename('./saved_models')))
print("Model Loaded")

### Inferencing the model 
Below code will be used for inferencing from the model and visualizing the results.

Here we do iteration from the `val_loader` and then select the class with highest probability and then compare it with actul class.

------------
* 모델 구조에서 log_softmax까진 입혀있었음.-> exponential

In [ ]:
# classes of fashion mnist dataset
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']
# creating iterator for iterating the dataset
dataiter = iter(val_loader)
images, labels = dataiter.next()
images_arr = []
labels_arr = []
pred_arr = []
# moving model to cpu for inference 
model.to("cpu")
# iterating on the dataset to predict the output

######### ke test
img, label = images[0], labels[0]
print(img.shape)    # torch.Size([1, 28, 28])
img = img.unsqueeze(0)  
label = label.item()  
print(img.shape)  # batch 차원을 임의로 한개 늘림  torch.Size([1, 1, 28, 28])
print(label)  # 1

pred_img = model(img)
print(pred_img.shape)  # torch.Size([1, 10])
print(pred_img)  # tensor([[-25.8288,   0.0000, -29.3317, ..  -28.9239]], grad_fn=<LogSoftmaxBackward0>)

pred_img = torch.exp(pred_img)  # [[3.7309e-01, 2.9141e-06, 5.7604e-01, 2.7260e-02, 2.0285e-02, .. ]]
print(pred_img.shape, pred_img) # (1, 10)
print(pred_img.sum())  # 1

pred_img = pred_img.data.numpy().squeeze()  # 한껍데이 벗겨냄
print(pred_img.shape) # (10, )

aa = np.argmax(pred_img)  # label index출력
print(aa)


for i in range(0,10):
    images_arr.append(images[i].unsqueeze(0))
    labels_arr.append(labels[i].item())
    ps = torch.exp(model(images_arr[i]))    # 모델 구조에서 log_softmax까진 입혀있었음.-> exponential
    ps = ps.data.numpy().squeeze()
    pred_arr.append(np.argmax(ps))
# plotting the results
fig = plt.figure(figsize=(25,4))
for i in range(10):
    ax = fig.add_subplot(2, 20/2, i+1, xticks=[], yticks=[])
    ax.imshow(images_arr[i].resize_(1, 28, 28).numpy().squeeze())
    ax.set_title("{} ({})".format(classes[pred_arr[i]], classes[labels_arr[i]]),
                 color=("green" if pred_arr[i]==labels_arr[i] else "red"))

### Refrences 
* [Pytorch Ignite Text CNN example notebook](https://github.com/pytorch/ignite/blob/master/examples/notebooks/TextCNN.ipynb)
* [Pytorch Ignite MNIST example](https://github.com/pytorch/ignite/blob/master/examples/mnist/mnist.py)